# Profitable App Profiles for the Apple Store and Google Play Markets

The purpose of this project is to analize applications information to understand the apps profiles people use and like the most, especially the free apps available in both Apple Store and Google Play markets. We want to have a better understanding of user preferences in order to develop profitable applications for both iOs and Android users.

This project is set as the final guided project to the Dataquest's Python Fundamentals course.

## Data preview

We are going to use for the analysis two datasets already available that contains the kind of information we need:

- [A dataset](https://www.kaggle.com/lava18/google-play-store-apps) from the Google Play Store, containing information about around 10,000 apps.
- [A dataset](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps) from the Apple Store, containing information about around 7,000 apps.

We begin with having a preview of the data. We needd to understant the structure of the data sets and the kind of data we are going to find.

We start by opening both datasets, read them and storing them in a variable to be explored and manipulated.

In [2]:
# Opening, reading and storig the Apple Dataset
apple_opened_file = open('AppleStore.csv', encoding = 'utf8')
from csv import reader
apple_read_file = reader(apple_opened_file)
apple_apps_data = list(apple_read_file)
apple_header = apple_apps_data[0]
apple_apps_data = apple_apps_data[1:]

# Opening, reading and storig the Google Dataset
google_opened_file = open('googleplaystore.csv', encoding = 'utf8')
google_read_file = reader(google_opened_file)
google_apps_data = list(google_read_file)
google_header = google_apps_data[0]
google_apps_data = google_apps_data[1:]

['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']


Number of rows: 7197
Number of columns: 16


In order to have a better look of the data, we are going to use the following `explore_data()` function:

In [52]:
# Function to explore the data en each dataset in a more readable way.
def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]    
    for row in dataset_slice:
        print(row)
        print('\n')

    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))


# Explore the dataset using the explore_data function.
explore_data(apple_apps_data, 0, 3, True)

# Explore the dataset using the explore_data function.
explore_data(google_apps_data, 0, 3, True)

To understand which columns of the datasets are useful for the goals of the project, we should undertand the data contained in each column. We can read the documentation of each dataset for better understanding of the information contained in each column. Here is documentation of the [Apple Store](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps) dataset and here the documentation of the [Google Play Store](https://www.kaggle.com/lava18/google-play-store-apps) dataset.

We can explore the header rows of each dataset to have a better look of the data:

In [4]:
# Header row of the Apple Store dataset.
print(apple_header,'\n')

# Header row of the Google Play Store dataset.
print(google_header, '\n')

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic'] 

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver'] 



First of all, for tracking purposes we need the app name column. Second of all, we are interested in analyze what kind of apps are more attractive to user, as we need to delevelop free applications that are mainly profitable via the amount of users. Given the context, the first column relevant for the analysis, in both datasets, is the 'price' column.

Other columns that can give valuable information are the 'user_rating' and 'Rating' columns (for the Apple and Google datasets). Another column that might be valuable for analysis purposes is the genre column, to identify which genres are more likeable.
Finally, the 'rating_count_tot' and 'Reviews' columns can provide valuable context about how many people is using each app.

## Data Cleaning

### Wrong data

In the discusion section of the Google Play dataset is a [dedicated discusion](https://www.kaggle.com/lava18/google-play-store-apps/discussion/66015) that points out the row 10472 of the dataset has a missing value and wrong data.
Cheking the discusion, we can notice that this is the index value for the dataset without the header row. Since we stored the header row in a diferent variable, it should be the same for us. We can check the problem with the row by printing it, along with the header row:

In [5]:
print(google_apps_data[10472])

['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']


As we can notice, the 'Category' column is missing in the row. By checking the number of columns of the row vrs the number of columns of the dataset we can verify that there's a missing column:

In [6]:
# Number of columns of the row.
print(len(google_apps_data[10472]))

# Number of columns of the dataset.
print(len(google_apps_data[0]))

12
13


Therefore, we proceed to remove the row from the dataset:

In [7]:
del(google_apps_data[10472]) # This shouldn't be run more than once or it will delete more rows than the one we wanted.

### Duplicates

Exploring the Google Play Store dataset, we realize there are few row that are duplicated. Here are some of the apps that are reapeated:

In [8]:
for row in google_apps_data:
    name = row[0]
    if name == 'Instagram':
        print(row)

['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577446', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66509917', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']


We can check how many duplicates are in the dataset by separating them from the rest of the data.

In [9]:
# Create an empty list to store duplicate and unique apps.
duplicate_apps = []
unique_apps = []

# Check what rows are duplicated.
for app in google_apps_data:
    name = app[0]
    if name in unique_apps:
        duplicate_apps.append(name)
    else:
        unique_apps.append(name)

# Print the number of duplicated apps.
print('Number of duplicate apps:', len(duplicate_apps))

Number of duplicate apps: 1181


We need to remove the duplicates, but instead of do it randomly, we need to find a better way to decide which duplicates are going to be deleted to maintain the dataset with the right data. In this case, given that each duplicated row has a diferent number of reviews, we will keep the row that has the highest number of reviews per app and delete the rest of the duplicates.

In [10]:
# Storing in a dictionary only the unique apps and the highest number of reviews per app.
reviews_max = {}

for row in google_apps_data:
    name = row[0]
    n_reviews = float(row[3])
    if name in reviews_max and reviews_max[name] < n_reviews:
        reviews_max[name] = n_reviews
    elif name not in reviews_max:
        reviews_max[name] = n_reviews

print(len(reviews_max))

9659


We will use the `reviews_max` dictionary to clean the dataset and create a new one, without the duplicates.
For this purpose, with need to do the following steps:

- Create two empty list. One to store the new dataset and the other to record wich app is already added to the dataset. 
- Loop through the original dataset. For each row, we store the name and the number of reviews in a variable. We first check that the number if reviews is the same that the one we store in the `reviews_max` dictionarys. Then, we check if the name of the app is not in the `already_added` list. We need to check this considering that some duplicate apps has the same number of reviews for each duplicated row. In this case, we still only want to add just one row to the cleaned dataset.
- If both conditions are met, then we add the entire row to the clean dataset and then store the name of the app in the `already_added` list.

In [11]:
# New list to store the clean dataset.
android_clean = []

# List to store app names already added to the cleaned dataset.
already_added = []

# Looping through the dataset.
for row in google_apps_data:
    name = row[0]
    n_reviews = float(row[3])
    
    if (n_reviews == reviews_max[name]) and (name not in already_added):
        android_clean.append(row)
        already_added.append(name)

We can use the `explore_data()` function to make sure the resulting `android_clean` dataset went as expected. The clean dataset should have 9659 final rows:

In [12]:
explore_data(android_clean, 0, 3, True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


Number of rows: 9659
Number of columns: 13


### Removing non-english apps

We want to focus our analysis on the apps directed to English speaking audience. However, if we continue exploring the dataset we realize there are apps with names that sugest they are not directed to an english speaking audience. Therefore, we want to remove this applications from the dataset before starting our analysis.

Each character has a codification number, used by the computer to 'translate' the character to it's oun code.
In case of the characters used in an English text, these are coded in the ASCII system. In ASCII, each character has a code from 0 to 127. The built-in function `ord()` allows us to check the code number asociated with each character. Hence, we can use this function to remove those apps that containg character in their names are not in the ASCII standard.

We need to create a function that check characters in a string and return `True`, if the characters are used in the English language, or `False`, if not.

In [13]:
# Building a function that receive a string and check if any character in the string does not belong to the ASCII standard.
def is_english(a_string):
    for char in a_string:
        if ord(char) > 127:
            return False
    return True

# Checking the function
print(is_english('Instagram'))
print(is_english('爱奇艺PPS -《欢乐颂2》电视剧热播'))
print(is_english('Docs To Go™ Free Office Suite'))
print(is_english('Instachat 😜'))
    

True
False
False
False


Checking the function, we can notice that it labels as non-English those strings that contains special characters, like emojis, but that actually are in English. This happens because this special characters are not encoded in ASCII, thus are outside the 127 number range. In that case, we are going to start verifying if more than 3 characters of each name don't satisfy the condition, to minimize the lost of valuable data.
Let's modify the previous function:

In [14]:
# Modified function
def is_english(a_string):
    rep = 0
    for char in a_string:
        if ord(char) > 127:
            rep += 1
    if rep > 3:
        return False
    else:
        return True

# Checking the modified function
print(is_english('Docs To Go™ Free Office Suite'))
print(is_english('Instachat 😜'))
print(is_english('爱奇艺PPS -《欢乐颂2》电视剧热播'))

True
True
False


We now use the `is_english()` function to filter each dataset:

In [15]:
# New dataset for the Apple Store data. 
apple_english = []
# New dataset for the Google Play Store data.
google_english = []

# Looping through the Apple Store dataset to separate the english apps.
for row in apple_apps_data:
    name = row[1]
    if is_english(name):
        apple_english.append(row)

# Looping through the Google Play Store dataset to separate the english apps. 
for row in android_clean:
    name = row[0]
    if is_english(name):
        google_english.append(row)

We can expore the resulting datasets to check that everyting went as expected:

In [16]:
# Exploring the new Apple Store dataset.
explore_data(apple_english, 0, 3, True)

['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']


Number of rows: 6183
Number of columns: 16


In [17]:
# Exploring the new Google Play Store dataset.
explore_data(google_english, 0, 3, True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


Number of rows: 9614
Number of columns: 13


### Isolating the free apps

As we said before, we are interested in those applications that are free to the users. Hence, we need to isolate the free apps in our data sets in order to analyze only those apps.

In [18]:
# Create new lists to store only the apps of each dataset that are free.
apple_free = []
google_free = []

# Looping through both datasets to separate free from non-free apps.
for row in apple_english:
    price = row[4]
    if price == '0.0':
        apple_free.append(row)

for row in google_english:
    price = row[7]
    if price == '0':
        google_free.append(row)


Let's check the length of the resulting datasets:

In [51]:
print('Length of the Apple Store data set:', len(apple_free))
print('Length of the Google Play Store data set:',len(google_free))

Length of the Apple Store data set: 3222
Length of the Google Play Store data set: 8864


## Data Analysis

### Most common apps by Genre

As mentioned in the introduction, our goal is develope an app that is highly atractive to a great numbre of users. The development and deployment process we want to follow has three main steps:

- Built a version for Andriod and release it in the Google Play Store.
- If the app has good response, develop it further.
- If the app is profitable after six months, built a iOs version and release it in the Apple Store.

Given that, we need to find app profiles that are atractive for both Android and iOs users.
To start our analysis, we can use the most common genres for each market. In the Android dataset, this information is in the `Category`and `Genres` column. In the iOs dataset, we have the `prime_genre` column.

Using this columns, we are going to create frequency tables to identify the most common genres for each dataset.
We begin writing a function to create the frequency tables:

In [20]:
# Function that receive a dataset and an index that indicates the column we want to obtain the frequency table. 
# Returns the frequency table with percentages per genre.
def freq_table(dataset, index):
    frequency_table = {}
    
    for row in dataset:
        value = row[index]
        if value in frequency_table:
            frequency_table[value] += 1
        else:
            frequency_table[value] = 1
    for key in frequency_table:
        valor = frequency_table[key]
        percentage = (valor/len(dataset)) *100
        frequency_table[key] = percentage
        
    return frequency_table

Then, we write a function to sort the frequency table to display the percentages in descending order:

In [21]:
# Function to display the pencentages per genre in descending order.

def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)

    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])

With both functions, we can display the frequency tables for the three columns we identify above and analyze it's results:

In [22]:
# Frequency table for the prime_genre column.
display_table(apple_free, 11)

Games : 58.16263190564867
Entertainment : 7.883302296710118
Photo & Video : 4.9658597144630665
Education : 3.662321539416512
Social Networking : 3.2898820608317814
Shopping : 2.60707635009311
Utilities : 2.5139664804469275
Sports : 2.1415270018621975
Music : 2.0484171322160147
Health & Fitness : 2.0173805090006205
Productivity : 1.7380509000620732
Lifestyle : 1.5828677839851024
News : 1.3345747982619491
Travel : 1.2414649286157666
Finance : 1.1173184357541899
Weather : 0.8690254500310366
Food & Drink : 0.8069522036002483
Reference : 0.5586592178770949
Business : 0.5276225946617008
Book : 0.4345127250155183
Navigation : 0.186219739292365
Medical : 0.186219739292365
Catalogs : 0.12414649286157665


The results show us that in the Apple Store dataset, for english speakers, the most common free apps are those related to games (58%). This is followed by apps related to Entertainment (7%), Photo & Video (4.96%) and finally Education and Social Networking (3.66% and 3.28). 
The most common free apps among english users are those dedicated to entertainment, while those for practical purposes have less users. However, we can't say from this column that the apps dedicated to entertainment are also those with the mayority of users just because they're the most common genre in the dataset.

In [23]:
# Frequency table for the Category column 
display_table(google_free, 1)

FAMILY : 18.907942238267147
GAME : 9.724729241877256
TOOLS : 8.461191335740072
BUSINESS : 4.591606498194946
LIFESTYLE : 3.9034296028880866
PRODUCTIVITY : 3.892148014440433
FINANCE : 3.7003610108303246
MEDICAL : 3.531137184115524
SPORTS : 3.395758122743682
PERSONALIZATION : 3.3167870036101084
COMMUNICATION : 3.2378158844765346
HEALTH_AND_FITNESS : 3.0798736462093865
PHOTOGRAPHY : 2.944494584837545
NEWS_AND_MAGAZINES : 2.7978339350180503
SOCIAL : 2.6624548736462095
TRAVEL_AND_LOCAL : 2.33528880866426
SHOPPING : 2.2450361010830324
BOOKS_AND_REFERENCE : 2.1435018050541514
DATING : 1.861462093862816
VIDEO_PLAYERS : 1.7937725631768955
MAPS_AND_NAVIGATION : 1.3989169675090252
FOOD_AND_DRINK : 1.2409747292418771
EDUCATION : 1.1620036101083033
ENTERTAINMENT : 0.9589350180505415
LIBRARIES_AND_DEMO : 0.9363718411552346
AUTO_AND_VEHICLES : 0.9250902527075812
HOUSE_AND_HOME : 0.8235559566787004
WEATHER : 0.8009927797833934
EVENTS : 0.7107400722021661
PARENTING : 0.6543321299638989
ART_AND_DESIGN : 

In [24]:
# Frequency table for the Genres column.
display_table(google_free, 9)

Tools : 8.449909747292418
Entertainment : 6.069494584837545
Education : 5.347472924187725
Business : 4.591606498194946
Productivity : 3.892148014440433
Lifestyle : 3.892148014440433
Finance : 3.7003610108303246
Medical : 3.531137184115524
Sports : 3.463447653429603
Personalization : 3.3167870036101084
Communication : 3.2378158844765346
Action : 3.1024368231046933
Health & Fitness : 3.0798736462093865
Photography : 2.944494584837545
News & Magazines : 2.7978339350180503
Social : 2.6624548736462095
Travel & Local : 2.3240072202166067
Shopping : 2.2450361010830324
Books & Reference : 2.1435018050541514
Simulation : 2.0419675090252705
Dating : 1.861462093862816
Arcade : 1.8501805054151623
Video Players & Editors : 1.7712093862815883
Casual : 1.7599277978339352
Maps & Navigation : 1.3989169675090252
Food & Drink : 1.2409747292418771
Puzzle : 1.128158844765343
Racing : 0.9927797833935018
Role Playing : 0.9363718411552346
Libraries & Demo : 0.9363718411552346
Auto & Vehicles : 0.9250902527075

In the Google Store dataset, in the other hand, the most common apps seems to be those dedicated to practical purpose, (education, business, lifestyle, productivity, etc). 
Compared to the results from the Apple Store dataset, the distribution of genres and categories in the Google Play Store data set is more uniform, even though there're categories that are significantly more common than the rest (family, games, tools), the different is less pronounced.

An important observation is that in the Apple Store dataset there is only one column that contains information about the clasification of the apps, while in the Google PLay Store dataset there are two columns. The last one seems to have mayor categories in the `Category` column and a more specific classification in the `Genres` columns.

Our conclusions about this review of columns, however, is that we need to do further analysis to identify the apps with more user, given the fact that this columns only provide the most common genres in the datasets, but not those with most users.

### Most Popular Apps by Genre on the App Store

To have a better understanding of the most popular apps among users, we are going to use the number of installs in the Google Store dataset. This is the best information on hand to establish which apps attracts more users.
We display the frequency table in descending order to have a better look of the most installed categories.

In [25]:
# Creating the frequency table of the Category column
freq_category = freq_table(google_free, 1)

In [48]:
avg_installs_google = []

for category in freq_category:
    total = 0
    len_category = 0
    for row in google_free:
        category_app = row[1]
        if category_app == category:
            n_installs = row[5]
            n_installs = n_installs.replace('+', '')
            n_installs = n_installs.replace(',', '')
            total += float(n_installs)
            len_category += 1
    category_avg = total / len_category
    avg_installs_google.append((category_avg, category))
sorted_avg_installs_google = sorted(avg_installs_google, reverse= True)
for entry in sorted_avg_installs_google:  
    print(entry[1], ':', entry[0])


COMMUNICATION : 38456119.167247385
VIDEO_PLAYERS : 24727872.452830188
SOCIAL : 23253652.127118643
PHOTOGRAPHY : 17840110.40229885
PRODUCTIVITY : 16787331.344927534
GAME : 15588015.603248259
TRAVEL_AND_LOCAL : 13984077.710144928
ENTERTAINMENT : 11640705.88235294
TOOLS : 10801391.298666667
NEWS_AND_MAGAZINES : 9549178.467741935
BOOKS_AND_REFERENCE : 8767811.894736841
SHOPPING : 7036877.311557789
PERSONALIZATION : 5201482.6122448975
WEATHER : 5074486.197183099
HEALTH_AND_FITNESS : 4188821.9853479853
MAPS_AND_NAVIGATION : 4056941.7741935486
FAMILY : 3695641.8198090694
SPORTS : 3638640.1428571427
ART_AND_DESIGN : 1986335.0877192982
FOOD_AND_DRINK : 1924897.7363636363
EDUCATION : 1833495.145631068
BUSINESS : 1712290.1474201474
LIFESTYLE : 1437816.2687861272
FINANCE : 1387692.475609756
HOUSE_AND_HOME : 1331540.5616438356
DATING : 854028.8303030303
COMICS : 817657.2727272727
AUTO_AND_VEHICLES : 647317.8170731707
LIBRARIES_AND_DEMO : 638503.734939759
PARENTING : 542603.6206896552
BEAUTY : 51315

As we can notice, communications, video players and social free applications are the most installed by Android users. However, we know that in this categories there are giant companies that have the majority of the market (Whatsapp, Skype, Facebook, Youtube, etc).

We need to check this results with the information in the Apple Store dataset:


In [47]:
# Create the frequency table of the prime_genre column.
freq_prime_genres = freq_table(apple_free, 11)

In [50]:
# Create a table with genres with most user count ratings.
avg_installs_apple = []

for category in freq_prime_genres:
    total = 0
    len_category = 0
    for row in apple_free:
        category_app = row[11]
        if category_app == category:
            rating_count = row[5]
            total += float(rating_count)
            len_category += 1
    category_avg = total / len_category
    avg_installs_apple.append((category_avg, category))
sorted_avg_installs_apple = sorted(avg_installs_apple, reverse= True)
for entry in sorted_avg_installs_apple:  
    print(entry[1], ':', entry[0])

Navigation : 86090.33333333333
Reference : 74942.11111111111
Social Networking : 71548.34905660378
Music : 57326.530303030304
Weather : 52279.892857142855
Book : 39758.5
Food & Drink : 33333.92307692308
Finance : 31467.944444444445
Photo & Video : 28441.54375
Travel : 28243.8
Shopping : 26919.690476190477
Health & Fitness : 23298.015384615384
Sports : 23008.898550724636
Games : 22788.6696905016
News : 21248.023255813954
Productivity : 21028.410714285714
Utilities : 18684.456790123455
Lifestyle : 16485.764705882353
Entertainment : 14029.830708661417
Business : 7491.117647058823
Education : 7003.983050847458
Catalogs : 4004.0
Medical : 612.0


Among iOs users, the most popular genre seems to be Navigation. However, this has the same isue we discused about the popular categories in Android users: there are few giants that already took the market (Waze and Google maps). The next genres are again Social Networking (Facebook, Twitter, etc) and Music (Youtube).

There are some other similar genres and categories similar in their positions among the most popular on both datasets. Those are, for iOs and Android users respectively, Books and Books_and Reference, Travel and Travel_and_Local and Shopping (in both datasets). The category that involve games has also some popularity in both datasets. This categories should have a better review as posible targets to develop new apps that can be attractive to both audiences of users of free applications. However, as we notice wen checked the most common apps, this category might be saturated.

## Conclusions

We analyzed the information about popular free apps for both iOs and Android users. We were looking for a category that might be attrative to both markets in order to further develop new free apps.
We concluded that there are some categories that have potential to have new offers and reach a good popularity among both audiences. This are those categories related to books, travel and shopping, but we need to do more analysis to determine the final targets of a posible new app.